In [1]:
try: 
    import cv2
    import torch
    import torchvision
    import sklearn.svm
except:
    %pip install opencv-python-headless==4.9.0.80
    %pip install torch
    %pip install torchvision
    %pip install torchsummary 

import torch
from torch.utils.data import Dataset
from torch import cuda
from torchvision import transforms, datasets, models
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

from pathlib import Path
from timeit import default_timer as timer
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter

from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import random
import numpy as np
import time
import copy

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


from torchsummary import summary
from PIL import Image

np.set_printoptions(threshold=np.inf)

print('import successful')

use_gpu = torch.cuda.is_available()

if use_gpu:
    print('gpu accessible')

torch.cuda.is_available()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 10.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
import successful
gpu accessible


True

In [33]:
# Data paths
EMOREACT = Path('EmoReact')
FER = Path('FER-2013')
KDEF = Path('KDEF-AKDEF')
NIMH = Path('NIMH-CHEFS')

# General paths
BASE_PATH = Path('/home/jovyan/work/data/out')
MODEL_PATH = Path('/home/jovyan/work/models')

# Set dataset here
DATA = NIMH

# Dataset-specific paths
CURRENT_PATH = BASE_PATH / DATA
LABELS = [f.name for f in CURRENT_PATH.iterdir() if f.is_dir()]
IMAGE_PATHS = list(CURRENT_PATH.rglob('*.jpg'))

# Constants for splitting dataset
TRAIN = 'train'
TEST = 'test'
VAL = 'val'

FEATURES = 'feature-extraction'
TRANSFER = 'transfer-learning'
FINETUNE = 'fine-tuning'

batch_size = 20

# CUDA
train_on_gpu = cuda.is_available()
print(f'[INFO] Train on gpu ...{train_on_gpu}')
if train_on_gpu:
    gpu_count = cuda.device_count()
    print(f'[INFO] {gpu_count} gpus detected.')
    if gpu_count > 1:
        multi_gpu = True
    else:
        multi_gpu = False

[INFO] Train on gpu ...True
[INFO] 2 gpus detected.


In [34]:
class Dataset(Dataset):
    def __init__(self, data_path, img_size, transforms=None, phase=TRAIN):
        self.data_path = Path(data_path / phase)
        self.img_size = img_size
        self.transform = transforms[phase]
        self.phase = phase

        self.classes = self._get_classes()
        self.image_paths = self._get_image_paths()
        self.num_classes = len(self.classes)

        self.class_idx = {class_name: idx for idx, class_name in enumerate(self.classes)}
        self.idx_class = {idx: class_name for class_name, idx in self.class_idx.items()}

        
    def _get_classes(self):
        return [f.name for f in (self.data_path).iterdir() if f.is_dir()]
    

    def _get_image_paths(self):
        paths = list(self.data_path.rglob('*.jpg'))
        random.shuffle( paths )
        return paths


    def __len__(self):
        return len(self.image_paths)
    

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx])
        img_path = self.image_paths[idx]

        if self.transform:
            img = self.transform(img)

        label = Path(img_path).parent.name

        return img, label
    

    def show_samples(self):
        fig = plt.figure(figsize=(20,20))

        for i in range(10):
            ax = fig.add_subplot(1, 10, i + 1)
            _, label = self.__getitem__(i)
            img_cv2 = self.get_cv2_img(i)

            ax.imshow(img_cv2, cmap='gray')
            ax.set_title(label)
            ax.axis('off')
        plt.show()


    def show_distribution(self):
        labels_count = Counter([self.__getitem__(i)[1] for i in tqdm(range(len(self.image_paths)))])
        sorted_counts = sorted(labels_count.items())
        labels, counts = zip(*sorted_counts)

        plt.figure(figsize=(10, 3))
        bars = plt.bar(labels, counts, color='skyblue')
        plt.xlabel(f'{DATA}')
        plt.ylabel('Count')
        plt.title('Counts per Emotion Category')
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)

        for bar, count in zip(bars, counts):
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5, count,
                    ha='center', va='bottom', color='black', fontsize=8) 

        plt.tight_layout()
        plt.show()

    def get_cv2_img(self, idx):
        img_path = self.image_paths[idx]
        return cv2.imread(str(img_path))
    

    def idx_to_class(self, idx_list):
        return [self.idx_class[idx] for idx in idx_list]


    def class_to_idx(self, class_list):
        return [self.class_idx[class_name] for class_name in class_list]
    
    
    def print_info(self):
        print(f"[INFO] Total number of images ...{len(self)}")
        print("[INFO] Number of classes: ", self.num_classes)
        print("[INFO] Classes: ", self.classes)

In [35]:
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

datasets = {
    x: Dataset(CURRENT_PATH, img_size=224, transforms=data_transforms, phase=x) for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {
    x : len(datasets[x]) for x in [TRAIN, VAL, TEST] 
}

n_classes = datasets[TRAIN].num_classes

In [36]:
vgg19 = models.vgg19(weights='IMAGENET1K_V1')
for param in vgg19.features.parameters():
    param.require_grad = False

num_features = vgg19.classifier[6].in_features

features = list(vgg19.classifier.children())[:-1]
features.extend([nn.Linear(num_features, n_classes)])
vgg19.classifier = nn.Sequential(*features)

if use_gpu:
    vgg19.cuda()

print("vgg19 init done")

vgg19 init done


In [37]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg19.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [38]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print(f"\rTraining batch {i}/{train_batches / 2}", flush=True)
                
            inputs = data[0]
            labels = torch.tensor(np.asarray(dataloaders[TRAIN].dataset.class_to_idx(data[1])))

            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()
            else:
                inputs, labels = inputs, labels
            
        
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            
            loss.backward()
            optimizer.step()

            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()

        avg_loss = loss_train / dataset_sizes[TRAIN]
        avg_acc = acc_train / dataset_sizes[TRAIN]

        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print(f"\rValidation batch {i}/{val_batches}", flush=True)
                
            inputs = data[0]
            labels = torch.tensor(np.asarray(dataloaders[TRAIN].dataset.class_to_idx(data[1])))
            
            if use_gpu:
                inputs, labels = inputs.cuda(),labels.cuda()
            else:
                inputs, labels = inputs, labels
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item()
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        
        print()
        print(f"Epoch {epoch} result: ")
        print(f"Avg loss (train): {avg_loss:.4f}")
        print(f"Avg acc (train): {avg_acc:.4f}")
        print(f"Avg loss (val): {avg_loss_val:.4f}")
        print(f"Avg acc (val): {avg_acc_val:.4f}")
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print(f"Training completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s")
    print(f"Best acc: {best_acc:.4f}")
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [39]:
vgg19 = train_model(vgg19, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=2)


Epoch 0/2
----------
Training batch 0/8.0


ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid 17696) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.

In [ ]:
torch.save(vgg19.state_dict(), 'VGG16_NIMH-CHEFS.pt')

In [11]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print(f"\rTest batch {i}/{test_batches}", flush=True)

        vgg.train(False)
        vgg.eval()
        inputs = data[0]
        labels = torch.tensor(np.asarray(dataloaders[TRAIN].dataset.class_to_idx(data[1])))

        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        else:
            inputs, labels = inputs, labels

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print(f"Evaluation completed in {elapsed_time // 60:.0f}m {elapsed_time % 60:.0f}s")
    print(f"Avg loss (test): {avg_loss:.4f}")
    print(f"Avg acc (test): {avg_acc:.4f}")
    print('-' * 10)


In [12]:
eval_model(vgg16, criterion)

Evaluating model
----------
Test batch 0/107
Test batch 100/107

Evaluation completed in 0m 45s
Avg loss (test): 1.6094
Avg acc (test): 0.2056
----------
